# Detection Evaluation

In [ ]:
%load_ext autoreload
%autoreload 2

from project_paths import paths 

from lane_detection_medium.inference import DetectionInference
from lane_detection_medium.evaluation import DetectionEvaluator

%matplotlib inline

## Trained Model Loading

In [ ]:
EXP_NAME = "train-2023-06-214"

CHECKPOINT_DPATH = paths.yolo_dpath / "LaneMarkingsDetection" / EXP_NAME / "weights"
MODEL_FPATH = CHECKPOINT_DPATH / "best.pt"

inference = DetectionInference.from_file(
    str(MODEL_FPATH), 
    device="cuda:0", 
    img_size=(640, 640)
)

## Evaluation

In [ ]:
data_dpath = paths.yolo_dpath / "data" / "2023_06_21" / "test"
evaluator = DetectionEvaluator(model=inference, verbose=True)

In [ ]:
evaluator.evaluate(data_dpath)